In [ ]:
!pip install rouge_score
!pip install bert_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=410fab1d79aca066fa74fd5b1d9868a468947fc22477678d8a16080b090586ce
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install evaluate
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def text_to_sentences(text):

    sentences = sent_tokenize(text)
    return sentences

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import evaluate
predictions = ["hello there general kenobi","foo bar foobar"]
references = [
     ["hello there general"],
     ["foo bar"] ]
bleu = evaluate.load('rouge') #evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'rouge1': 0.8285714285714285, 'rouge2': 0.7333333333333334, 'rougeL': 0.8285714285714285, 'rougeLsum': 0.8285714285714285}


In [ ]:
import pandas as pd
data = pd.read_csv('/content/data_u3.csv')


In [ ]:
i=0
predictions = [ data[ 'Direct_propting_gpt-3.5-turbo'][i]]
references = [text_to_sentences(data['Abstract'][i] ) ]
bleu = evaluate.load('rouge') #evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
results['rouge1']

0.2222222222222222

In [ ]:
data.columns

Index(['Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3',
       'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'Community', 'Abstract',
       'Terms', 'frequent_words', 'Direct_propting_gpt-3.5-turbo',
       'Direct_propting_gpt-4o', 'zeroshot_COT_gpt-3.5-turbo',
       'self_consistency_COT_gpt3.5-turbo', 'self_feedback_gpt3.5-turbo',
       'self-critique', 'Llama-2-7B-Chat', 'microsoft_phi', 'Llama_3',
       'Gemma_7', 'Mistral'],
      dtype='object')

In [ ]:
columns_=['Direct_propting_gpt-3.5-turbo',
       'Direct_propting_gpt-4o', 'zeroshot_COT_gpt-3.5-turbo',
       'self_consistency_COT_gpt3.5-turbo', 'self_feedback_gpt3.5-turbo','self-critique']

columns_=['Llama-2-7B-Chat', 'microsoft_phi', 'Llama_3',
       'Gemma_7', 'Mistral']

new_data=pd.DataFrame(columns=columns_)


In [ ]:

bleu = evaluate.load('rouge') #evaluate.load("bleu")

for j in columns_:
  x=[]
  for i in data.index :
    try:
      predictions = [ data[ j][i]]
      references = [text_to_sentences(data['Abstract'][i] ) ]
      results = bleu.compute(predictions=predictions, references=references)
      x.append(results['rouge1'])
    except:
      pass
  new_data.at['blue',j]= sum(x)/len(x)


In [ ]:
new_data

,Llama-2-7B-Chat,microsoft_phi,Llama_3,Gemma_7,Mistral
blue,0.13331,0.155866,0.234379,0.16436,0.241459


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')


In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
for j in columns_:
  x=[]
  for i in data.index :
    try:
      predictions = str(data[ j][i])
      references = str(data['Abstract'][i] )
      scores = scorer.score(predictions,
                      references)
      x.append(scores['rougeL'].fmeasure)
    except:
      pass
  new_data.at['rougeL',j]= sum(x)/len(x)

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there general kenobi"]
references = [ "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
results

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.8915876150131226],
 'recall': [0.949322521686554],
 'f1': [0.9195497035980225],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.42.4)'}

In [ ]:
for j in columns_:
  x=[]
  for i in data.index :
    try:
        predictions = [ str(data[ j][i])]
        references = [str(data['Abstract'][i] ) ]
        results = bertscore.compute(predictions=predictions, references=references, lang="en")
        x.append(results['f1'][0])
    except:
      pass
  new_data.at['bertscore',j]= sum(x)/len(x)

In [ ]:
new_data

,Llama-2-7B-Chat,microsoft_phi,Llama_3,Gemma_7,Mistral
blue,0.13331,0.155866,0.234379,0.16436,0.241459
rougeL,0.034962,0.049108,0.09909,0.053606,0.117147
bertscore,0.775696,0.795181,0.795516,0.793534,0.808583


# Move score

In [ ]:
!git clone https://github.com/Jaber-Valinejad/emnlp19-moverscore
!pip install torch>=1.0.0 pyemd==0.5.1 pytorch-transformers>=1.1.0
%cd emnlp19-moverscore/
from moverscore_v2 import get_idf_dict, word_mover_score
from collections import defaultdict

Cloning into 'emnlp19-moverscore'...
remote: Enumerating objects: 465, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 465 (delta 35), reused 13 (delta 3), pack-reused 400
Receiving objects: 100% (465/465), 7.07 MiB | 6.84 MiB/s, done.
Resolving deltas: 100% (218/218), done.
/content/QuestEval/emnlp19-moverscore


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
'''
example 1:
from moverscore_v2 import plot_example
reference = 'they are now equipped with air conditioning and new toilets.'
translation = 'they have air conditioning and new toilets.'
plot_example(True, reference, translation)

'''
translations=['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']
references=['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.']
idf_dict_hyp = get_idf_dict(translations) # idf_dict_hyp = defaultdict(lambda: 1.)
idf_dict_ref = get_idf_dict(references) # idf_dict_ref = defaultdict(lambda: 1.)
scores = word_mover_score(references, translations, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
scores

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[0.8845664256074734, 0.6582527863649539, 0.61154118148405]

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')  # Download the necessary resources for sentence tokenization
from nltk.tokenize import sent_tokenize

# Example text
text = "Hello, world! This is an example text. It contains multiple sentences. Let's tokenize it."

# Convert text to a list of sentences
sentences = sent_tokenize(text)

# Print the list of sentences
print(sentences)

['Hello, world!', 'This is an example text.', 'It contains multiple sentences.', "Let's tokenize it."]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import numpy as np
np.mean([3,4,5,4])

4.0

In [ ]:
for j in columns_:
  x=[]
  print(j)
  for i in data.index :
    try:
         translations=sent_tokenize(str(data[ j][i]))
         references=sent_tokenize(str(data['Abstract'][i] ) )
         idf_dict_hyp = get_idf_dict(translations) # idf_dict_hyp = defaultdict(lambda: 1.)
         idf_dict_ref = get_idf_dict(references) # idf_dict_ref = defaultdict(lambda: 1.)
         scores = word_mover_score(references, translations, idf_dict_ref, idf_dict_hyp, \
                          stop_words=[], n_gram=1, remove_subwords=True)
         x.append(np.mean(scores))
    except:
      pass
  new_data.at['moverscore_v2',j]= sum(x)/len(x)

Llama-2-7B-Chat


Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors


microsoft_phi


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Llama_3
Gemma_7
Mistral


In [ ]:
new_data

,Llama-2-7B-Chat,microsoft_phi,Llama_3,Gemma_7,Mistral
blue,0.13331,0.155866,0.234379,0.16436,0.241459
rougeL,0.034962,0.049108,0.09909,0.053606,0.117147
bertscore,0.775696,0.795181,0.795516,0.793534,0.808583
moverscore_v2,1.0,1.0,0.483837,0.478194,0.690979


# QuestEval

https://github.com/ThomasScialom/QuestEval/tree/main

In [ ]:
!pip install torch pytest transformers sentencepiece datasets bert_score Unidecode spacy
!git clone https://github.com/ThomasScialom/QuestEval
%cd QuestEval/

fatal: destination path 'QuestEval' already exists and is not an empty directory.
/content/QuestEval


In [ ]:
from questeval.questeval_metric import QuestEval
questeval = QuestEval(no_cuda=True)

source_1 = "Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of 35000."
prediction_1 = "Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal."
references_1 = [
    "Since 2000, the recipient of the Kate Greenaway Medal will also receive the Colin Mears Awad which worth 5000 pounds",
    "Since 2000, the recipient of the Kate Greenaway Medal has also been given the Colin Mears Award."
]

source_2 = "He is also a member of another Jungiery boyband 183 Club."
prediction_2 = "He also has another Jungiery Boyband 183 club."
references_2 = [
    "He's also a member of another Jungiery boyband, 183 Club.",
    "He belonged to the Jungiery boyband 183 Club."
]

score = questeval.corpus_questeval(
    hypothesis=[prediction_1, prediction_2],
    sources=[source_1, source_2],
    list_references=[references_1, references_2]
)

print(score)

/content/QuestEval/questeval/questeval_metric.py:106: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.metric_BERTScore = load_metric("bertscore")
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the max

{'corpus_score': 0.6332757670524312, 'ex_level_scores': [0.5924053125083446, 0.6741462215965177]}


In [ ]:
source_1 = "Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of 35000."
prediction_1 = "Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal."
score = questeval.corpus_questeval(
    hypothesis=[prediction_1],
    sources=[source_1],)
score['corpus_score']

0.5845596884687743

In [ ]:
import pandas as pd
data = pd.read_csv('/content/data_u3.csv')
new_data=pd.read_csv('/content/prpmp_perfomance1.csv')

In [ ]:
columns_=['Direct_propting_gpt-3.5-turbo',
       'Direct_propting_gpt-4o', 'zeroshot_COT_gpt-3.5-turbo',
       'self_consistency_COT_gpt3.5-turbo', 'self_feedback_gpt3.5-turbo','self-critique']
columns_=['Llama-2-7B-Chat', 'microsoft_phi', 'Llama_3',
       'Gemma_7', 'Mistral']

In [ ]:
for j in columns_:
  x=[]
  print(j)
  for i in data.index[:10] :
    try:
         source_1 = str(data[ j][i])
         prediction_1 =str(data['Abstract'][i] )
         score = questeval.corpus_questeval(hypothesis=[prediction_1], sources=[source_1],)
         x.append(score['corpus_score'])
    except:
      pass
  new_data.at['QuestEval',j]= sum(x)/len(x)

Llama-2-7B-Chat
microsoft_phi


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Llama_3
Gemma_7
Mistral


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [ ]:
display(new_data)

#new_data.to_csv('prpmp_perfomance2.csv')

,Unnamed: 0.1,Unnamed: 0,Direct_propting_gpt-3.5-turbo,Direct_propting_gpt-4o,zeroshot_COT_gpt-3.5-turbo,self_consistency_COT_gpt3.5-turbo,self_feedback_gpt3.5-turbo,self-critique,Llama-2-7B-Chat,microsoft_phi,Llama_3,Gemma_7,Mistral
0,0,blue,0.169815,0.148953,0.182766,0.182014,0.233695,0.174865,NaN,NaN,NaN,NaN,NaN
1,1,rougeL,0.054511,0.053996,0.058214,0.056834,0.078002,0.054632,NaN,NaN,NaN,NaN,NaN
2,2,bertscore,0.800486,0.797543,0.800410,0.801813,0.805295,0.793827,NaN,NaN,NaN,NaN,NaN
3,3,moverscore_v2,0.742506,0.493061,0.743867,0.615152,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
4,blanc,NaN,0.040676,0.039954,0.047378,0.042787,0.063025,0.038936,NaN,NaN,NaN,NaN,NaN
blanc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023862,0.041230,0.054672,0.044103,0.050295
QuestEval,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.302520,0.313798,0.372101,0.332544,0.295536


# BlanC

In [ ]:
! pip install torch==2.3.1
!pip install blanc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.3.1+cu121
    Uninstalling torch-2.3.1+cu121:
      Successfully uninstalled torch-2.3.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.1+cu121 requires torch==2.3.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.18.0 requires torch>=2.3.0, but you have torch 1.13.1 which is incompatible.
torchvision 0.18.1+cu121 requires torch==2.3.1, but you have torch 1.13.1 w

In [ ]:
! pip install torch==2.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
blanc 0.3.4 requires torch<2.0,>=1.0, but you have torch 2.3.1 which is incompatible.


In [ ]:
import nltk
# Download the punkt tokenizer
nltk.download('punkt')
from blanc import BlancHelp, BlancTune
document = "Jack drove his minivan to the bazaar to purchase milk and honey for his large family."
summary = "Jack bought milk and honey."
blanc_help = BlancHelp( inference_batch_size=128)
blanc_tune = BlancTune(inference_batch_size=24, finetune_mask_evenly=False, finetune_batch_size=24)
#blanc_help = BlancHelp()
#blanc_tune = BlancTune(finetune_mask_evenly=False, show_progress_bar=False)
blanc_help.eval_once(document, summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t

0.2222222222222222

In [ ]:
import pandas as pd
data = pd.read_csv('/content/data_u3.csv')
new_data=pd.read_csv('/content/prompt_performance.csv')

In [ ]:
columns_=['Direct_propting_gpt-3.5-turbo',
       'Direct_propting_gpt-4o', 'zeroshot_COT_gpt-3.5-turbo',
       'self_consistency_COT_gpt3.5-turbo', 'self_feedback_gpt3.5-turbo','self-critique']

columns_=['Llama-2-7B-Chat', 'microsoft_phi', 'Llama_3',
       'Gemma_7', 'Mistral']
for j in columns_:
  x=[]
  print(j)
  for i in data.index :
    try:
        document =str(data['Abstract'][i] )
        summary = str(data[ j][i])
        res=blanc_help.eval_once(document, summary)
        x.append(res)
    except:
      pass
  new_data.at['blanc',j]= sum(x)/len(x)

Llama-2-7B-Chat


100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


microsoft_phi


100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


Llama_3


100%|██████████| 1/1 [00:01<00:00,  1.93s/it]


Gemma_7


100%|██████████| 1/1 [00:01<00:00,  1.85s/it]


Mistral


100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


In [ ]:
new_data

,Unnamed: 0.1,Unnamed: 0,Direct_propting_gpt-3.5-turbo,Direct_propting_gpt-4o,zeroshot_COT_gpt-3.5-turbo,self_consistency_COT_gpt3.5-turbo,self_feedback_gpt3.5-turbo,self-critique,Llama-2-7B-Chat,microsoft_phi,Llama_3,Gemma_7,Mistral
0,0,blue,0.169815,0.148953,0.182766,0.182014,0.233695,0.174865,NaN,NaN,NaN,NaN,NaN
1,1,rougeL,0.054511,0.053996,0.058214,0.056834,0.078002,0.054632,NaN,NaN,NaN,NaN,NaN
2,2,bertscore,0.800486,0.797543,0.800410,0.801813,0.805295,0.793827,NaN,NaN,NaN,NaN,NaN
3,3,moverscore_v2,0.742506,0.493061,0.743867,0.615152,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
4,blanc,NaN,0.040676,0.039954,0.047378,0.042787,0.063025,0.038936,NaN,NaN,NaN,NaN,NaN
blanc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023862,0.04123,0.054672,0.044103,0.050295


In [ ]:
new_data.to_csv('LLM_performance1.csv')